In [3]:
import pandas as pd
from conexao_oracle import conexao_local_host
import oracledb

#   ABERTURA DO ARQUIVO
dados = pd.read_csv('tbl_cep_201805_csv.csv',encoding="ISO-8859-1")

In [4]:
dados_filtrados = dados[['estado','cidade','logradouro','cep','tipo']]
logradouros_filtrados = dados_filtrados.dropna(subset=['cep','logradouro'])

In [5]:

logradouros_filtrados



,estado,cidade,logradouro,cep,tipo
0,SP,São Paulo,Praça da Sé,1001000,Praça
1,SP,São Paulo,Praça da Sé,1001001,Praça
2,SP,São Paulo,Rua Filipe de Oliveira,1001010,Rua
3,SP,São Paulo,"Praça da Sé, 108",1001900,Praça
4,SP,São Paulo,"Praça da Sé, 371",1001901,Praça
...,...,...,...,...,...
1108496,RS,Ciríaco,"Avenida Dezenove de Maio, 743",99970970,Avenida
1108498,RS,São João Bosco (Ciríaco),"Rua Principal, s/n",99975970,Rua
1108501,RS,David Canabarro,"Rua Adelino Gazzoni, 160",99980970,Rua
1108502,RS,David Canabarro,"Distrito Capela São José do Capingui, s/n",99980974,Distrito


In [1]:

conexao = conexao_local_host()
with conexao.cursor() as cursor:
    l = 0
    for  linha in logradouros_filtrados.itertuples(index=False):
        cursor.callproc('SP_INSERIR_LOGRADOURO',[linha[0],linha[1],linha[2],linha[3],linha[4]])
        print(l,linha)
        conexao.commit(linha[0],linha[1],linha[2],linha[3],linha[4])
        l+=1
        

NameError: name 'conexao_local_host' is not defined

In [ ]:
lista_registros = []
conexao = conexao_local_host()
with conexao.cursor() as cursor:
    for index,linha in logradouros_filtrados.iterrows():
        fnc_pid_cidade = cursor.callfunc("FNC_FK_PID_CIDADE",int,[linha[1]])
        print(index,linha[0],fnc_pid_cidade,linha[2],linha[3],linha[4])


        if fnc_pid_cidade == None:
            print('valor nulo',index,linha) 
        else:
            lista_registros.append((linha[0],fnc_pid_cidade,linha[2],linha[3],linha[4]))


#cursor.executemany("INSERT INTO TB_LOGRADOURO (SIGLA_UF,PID_CIDADE,LOGRADOURO,CEP,TIPO) VALUES (:SIGLA_UF,:PID_CIDADE,:LOGRADOURO,:CEP,:TIPO)", lista_registros)
#conexao.commit

In [ ]:
cursor.executemany("INSERT INTO TB_LOGRADOURO (SIGLA_UF,PID_CIDADE,LOGRADOURO,CEP,TIPO) VALUES (:SIGLA_UF,:PID_CIDADE,:LOGRADOURO,:CEP,:TIPO)",lista_registros,batcherrors=True)
for error in cursor.getbatcherrors():
    print("Error", error.message, "at row offset", error.offset)

In [79]:
df_logradouro = pd.DataFrame(lista_registros)
df_logradouro

,0,1,2,3,4
0,SP,19,Rua dos Guarantãs,7437000,Rua
1,SP,19,Rua dos Jacarandás,7437005,Rua
2,SP,19,Rua dos Figueiras,7437010,Rua
3,SP,19,Rua dos Pinheiros,7437025,Rua
4,SP,19,Rua dos Cedros,7437050,Rua
...,...,...,...,...,...
75,SP,19,Rua Abatiá,7443015,Rua
76,SP,19,Rua Atalaia,7443020,Rua
77,SP,19,Rua Boa Esperança,7443025,Rua
78,SP,19,Rua Califórnia,7443030,Rua


In [ ]:
lista_registros = []
for index,linha in logradouros_filtrados.iterrows():
    conexao = conexao_local_host()
    cursor = conexao.cursor()
    fnc_pid_cidade = cursor.callfunc("FNC_FK_PID_CIDADE",int,[linha[1]])

    if fnc_pid_cidade is not None:
        lista_registros.append((linha[0],fnc_pid_cidade,linha[2],linha[3],linha[4]))
        if len(lista_registros) % 100 == 0:

            try:
                print('TOTAL DE REGISTROS A SER INSERIDOS EM TB_LOGRADOURO:',len(lista_registros))

                sql = """INSERT INTO TB_LOGRADOURO (SIGLA_UF,PID_CIDADE,LOGRADOURO,CEP,TIPO)  
                VALUES (:SIGLA_UF,:PID_CIDADE,:LOGRADOURO,:CEP,:TIPO)"""

                cursor.executemany(sql, lista_registros)
                conexao.commit()
                lista_registros = []
                
            except oracledb.IntegrityError:
                print(lista_registros)
                lista_registros = []
            except:
                print(lista_registros)
                lista_registros = []
                

if lista_registros:
    cursor.executemany(sql, lista_registros)
    conexao.commit()
